In [1]:
import json
import nltk
import re
import numpy as np
import tensorflow as tf
from prepro import convert_idx, save

In [2]:
with open('/home/soham/data/squad/sample1k-HCVerifyAll.json', 'r') as f:
    foo = json.load(f)

In [3]:
meh = []

In [4]:
foo['version']

'1.1'

In [5]:
# i = 0; j = 0
# original_paragraphs = {}
# adversarial_paragraphs = {}
# titles = set()

# def strip_id(id_):
#     return id_.split('-')[0]

# for datum in foo['data']:
#     titles.add(datum['title'])
    
#     for paragraph in datum['paragraphs']:
#         context = paragraph['context']
        
#         for qa in paragraph['qas']:
#             answers = qa['answers']
#             question = qa['question']
#             id_ = qa['id']
            
#             if strip_id(id_) == id_:
#                 assert id_ not in original_paragraphs
#                 original_paragraphs[id_] = (id_, datum['title'], context, question, answers)
#                 i += 1
#             else:
#                 if strip_id(id_) not in adversarial_paragraphs:
#                     adversarial_paragraphs[strip_id(id_)] = []
                
#                 #### YOU NEED TO UNCOMMENT THIS!! #####
#                 adversarial_paragraphs[strip_id(id_)] += [(context, question, answers)]
#                 j += 1

# print(i, j)

In [6]:
# np.random.seed(1337)
# train_titles = set(np.random.choice(sorted(list(titles)), size=int(len(titles)*0.7), replace=False))
# test_titles = titles - train_titles

In [7]:
# print(', '.join(sorted(list(train_titles))))
# print()
# print(', '.join(sorted(list(test_titles))))

3560 total:  

* 2560 mutated
* 1000 original

In [8]:
# np.random.seed(1337)

In [9]:
# train_pairs = []
# test_pairs = []

# for key, (id_, title, orig_para, orig_ques, orig_ans) in original_paragraphs.items():
#      #id_, title, orig_para, orig_ques, orig_answer = id_title_para_ques_ans
    
#     if title in train_titles:
#         train_pairs += [(id_, orig_para, orig_ques, orig_ans, None, 'orig')]
#     else:
#         assert title in test_titles
#         test_pairs += [(id_, orig_para, orig_ques, orig_ans, None, 'orig')]
    
#     if key in adversarial_paragraphs:
#         for j, (para, ques, ans) in enumerate(adversarial_paragraphs[key]):
#             injection = para[len(orig_para)+1:]
            
#             orig_sents = nltk.sent_tokenize(orig_para)
            
#             i = np.random.randint(0, len(orig_sents) + 1)
#             adv_para = ' '.join(orig_sents[:i] + [injection] + orig_sents[i:])
            
#             if i == 0:
#                 start = 0
#             else:
#                 start = len(' '.join(orig_sents[:i])) + 1
            
#             end = start + len(injection)
            
#             adv_ans = []
            
#             for answer in ans:
#                 answer_start = answer['answer_start'] + (end-start+1)
                
#                 if answer['answer_start'] > start:
#                     while not adv_para[answer_start:].startswith(answer['text']):
#                         answer_start += 1
                    
#                     assert adv_para[answer_start:].startswith(answer['text'])
                
#                 adv_ans += [{'answer_start': answer_start,
#                             'text': answer['text']}]
            
#             if title in train_titles:
#                 if np.random.randint(3, size=1)[0] == 1:
#                     train_pairs += [('%s-mut%d' % (id_, j), adv_para, orig_ques, adv_ans, (start, end), 'adv')]
#             else:
#                 assert title in test_titles
#                 if np.random.randint(3, size=1)[0] == 1:
#                     test_pairs += [('%s-mut%d' % (id_, j), adv_para, orig_ques, adv_ans, (start, end), 'adv')]                    

In [10]:
train_pairs = np.load('final_train_pairs.npy').tolist()
test_pairs = np.load('final_test_pairs.npy').tolist()

In [11]:
adv_count = lambda x: sum(map(lambda y: 1 if y[-2] == 'adv' else 0, x))

In [12]:
print('Train:', '%d,' % len(train_pairs), adv_count(train_pairs), 'of which are adversarial')
print('Test:', '%d,' % len(test_pairs), adv_count(test_pairs), 'of which are adversarial')

Train: 2178, 1570 of which are adversarial
Test: 1381, 989 of which are adversarial


In [13]:
import spacy
nlp = spacy.blank("en")

In [14]:
def word_tokenize(sent):
    doc = nlp(sent)
    return [token.text for token in doc]

In [15]:
def _get_word(word):
    for each in (word, word.lower(), word.capitalize(), word.upper()):
        if each in word2idx_dict:
            return word2idx_dict[each]
    return 1

def _get_char(char):
    if char in char2idx_dict:
        return char2idx_dict[char]
    return 1

In [16]:
with open('data/word2idx.json', 'r') as fh:
    word2idx_dict = json.load(fh)

with open('data/char2idx.json', 'r') as fh:
    char2idx_dict = json.load(fh)

In [17]:
para_limit = 1000
ques_limit = 100
char_limit = 16

In [18]:
import random
random.seed(1337)
random.shuffle(train_pairs)
random.shuffle(test_pairs)

In [19]:
len([tp[0] for tp in train_pairs]), len(set([tp[0] for tp in train_pairs]))

(2178, 2178)

In [20]:
%rm data/og_check_*.tf

In [21]:
writer = tf.python_io.TFRecordWriter('data/og_check_train.tf')

for id_, (id_but_ignore, context, ques, ans, start_end, tag, UNK) in enumerate(train_pairs):
    context = context.replace("''", '" ').replace("``", '" ')
    context_tokens = word_tokenize(context)
    context_chars = [list(token) for token in context_tokens]

    ques = ques.replace("''", '" ').replace("``", '" ')
    ques_tokens = word_tokenize(ques)
    ques_chars = [list(token) for token in ques_tokens]

    context_idxs = np.zeros([para_limit], dtype=np.int32)
    context_char_idxs = np.zeros([para_limit, char_limit], dtype=np.int32)
    ques_idxs = np.zeros([ques_limit], dtype=np.int32)
    ques_char_idxs = np.zeros([ques_limit, char_limit], dtype=np.int32)

    for i, token in enumerate(context_tokens):
        context_idxs[i] = _get_word(token)

    for i, token in enumerate(ques_tokens):
        ques_idxs[i] = _get_word(token)

    for i, token in enumerate(context_chars):
        for j, char in enumerate(token):
            if j == char_limit:
                break
            context_char_idxs[i, j] = _get_char(char)

    for i, token in enumerate(ques_chars):
        for j, char in enumerate(token):
            if j == char_limit:
                break

            ques_char_idxs[i, j] = _get_char(char)
    
    ### NEW PROC GOES HERE
    bad_y1 = np.zeros([para_limit], dtype=np.float32)
    bad_y2 = np.zeros([para_limit], dtype=np.float32)
    
    spans = convert_idx(context, context_tokens)
    
    if start_end is None:
        bad_y1[0] = 1
        bad_y2[0] = 1
    else:
        start, end = start_end
        
        answer_span = []
        
        for idx, span in enumerate(spans):
            if not (end <= span[0] or start >= span[1]):
                answer_span.append(idx)
                
        bad_y1[answer_span[0]] = 1
        bad_y2[answer_span[-1]] = 1
        
    ### NEW PROC GOES HERE
    y1 = np.zeros([para_limit], dtype=np.float32)
    y2 = np.zeros([para_limit], dtype=np.float32)
    
    start, end = ans[-1]['answer_start'], ans[-1]['answer_start'] + len(ans[-1]['text'])

    answer_span = []

    for idx, span in enumerate(spans):
        if not (end <= span[0] or start >= span[1]):
            answer_span.append(idx)

    y1[answer_span[0]] = 1
    y2[answer_span[-1]] = 1
    
    record = tf.train.Example(features=tf.train.Features(feature={
                              "context_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[context_idxs.tostring()])),
                              "ques_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[ques_idxs.tostring()])),
                              "context_char_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[context_char_idxs.tostring()])),
                              "ques_char_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[ques_char_idxs.tostring()])),
                              "y1": tf.train.Feature(bytes_list=tf.train.BytesList(value=[y1.tostring()])),
                              "y2": tf.train.Feature(bytes_list=tf.train.BytesList(value=[y2.tostring()])),
                              "id": tf.train.Feature(int64_list=tf.train.Int64List(value=[id_])),
                              }))
    
    writer.write(record.SerializeToString())

writer.close()

In [22]:
writer = tf.python_io.TFRecordWriter('data/og_check_test.tf')
meta_info = {}

for id_, (id_but_ignore, context, ques, ans, start_end, tag, UNK) in enumerate(test_pairs):
    context = context.replace("''", '" ').replace("``", '" ')
    context_tokens = word_tokenize(context)
    context_chars = [list(token) for token in context_tokens]

    ques = ques.replace("''", '" ').replace("``", '" ')
    ques_tokens = word_tokenize(ques)
    ques_chars = [list(token) for token in ques_tokens]

    context_idxs = np.zeros([para_limit], dtype=np.int32)
    context_char_idxs = np.zeros([para_limit, char_limit], dtype=np.int32)
    ques_idxs = np.zeros([ques_limit], dtype=np.int32)
    ques_char_idxs = np.zeros([ques_limit, char_limit], dtype=np.int32)

    for i, token in enumerate(context_tokens):
        context_idxs[i] = _get_word(token)

    for i, token in enumerate(ques_tokens):
        ques_idxs[i] = _get_word(token)

    for i, token in enumerate(context_chars):
        for j, char in enumerate(token):
            if j == char_limit:
                break
            context_char_idxs[i, j] = _get_char(char)

    for i, token in enumerate(ques_chars):
        for j, char in enumerate(token):
            if j == char_limit:
                break

            ques_char_idxs[i, j] = _get_char(char)
    
    ### NEW PROC GOES HERE
    bad_y1 = np.zeros([para_limit], dtype=np.float32)
    bad_y2 = np.zeros([para_limit], dtype=np.float32)
    
    spans = convert_idx(context, context_tokens)
    
    if start_end is None:
        bad_y1[0] = 1
        bad_y2[0] = 1
    else:
        start, end = start_end
        
        answer_span = []
        
        for idx, span in enumerate(spans):
            if not (end <= span[0] or start >= span[1]):
                answer_span.append(idx)
                
        bad_y1[answer_span[0]] = 1
        bad_y2[answer_span[-1]] = 1
    
    ### NEW PROC GOES HERE
    y1 = np.zeros([para_limit], dtype=np.float32)
    y2 = np.zeros([para_limit], dtype=np.float32)
    
    start, end = ans[-1]['answer_start'], ans[-1]['answer_start'] + len(ans[-1]['text'])

    answer_span = []

    for idx, span in enumerate(spans):
        if not (end <= span[0] or start >= span[1]):
            answer_span.append(idx)

    y1[answer_span[0]] = 1
    y2[answer_span[-1]] = 1
    
    if id_but_ignore in set([foo['uuid'] for foo in meta_info.values()]):
        print('hey', id_but_ignore)
        break
    
    meta_info[id_] = {'answers': [a['text'] for a in ans], 'question': ques, 'spans': spans, 'context': context, 'uuid': id_but_ignore}
    
    record = tf.train.Example(features=tf.train.Features(feature={
                              "context_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[context_idxs.tostring()])),
                              "ques_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[ques_idxs.tostring()])),
                              "context_char_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[context_char_idxs.tostring()])),
                              "ques_char_idxs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[ques_char_idxs.tostring()])),
                              "y1": tf.train.Feature(bytes_list=tf.train.BytesList(value=[y1.tostring()])),
                              "y2": tf.train.Feature(bytes_list=tf.train.BytesList(value=[y2.tostring()])),
                              "id": tf.train.Feature(int64_list=tf.train.Int64List(value=[id_])),
                              }))
    
    writer.write(record.SerializeToString())

writer.close()
save('data/og_check_test_meta.json', meta_info, message='meta_info')

Saving meta_info...


In [25]:
len(set([a for a in map(lambda x: x['uuid'], meta_info.values())]))

1381